In [9]:
from scripts.software_frs import frs_traceability2
FRS = frs_traceability2
from officelib.xllib import *
from officelib.const import xlconst as xlc
import sys
import os
import re

from scripts.tools.issuetracker import IssuetrackerAPI
from datetime import datetime

import jpnotebooks.Software.SDLC_traceability_tools.issuetracker_item_extracter as iie

In [10]:
_cache = None
_age = None
def _download_issues():
    api = IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
    return api.download_issues("pbssoftware", status_id="*")

def get_issues():
    global _cache, _age
    if not _cache or (datetime.now() - _age).total_seconds() > (8*3600):  # 8 hr
        _cache = _download_issues()
        _age = datetime.now()
    return _cache

In [11]:
def filter_relevant_issues(issues, rel_cb):
    relevant = {}
    for v in issues.values():
        if rel_cb(v):
                relevant[v.id] = v
    return relevant

def load_frs_from_issuetracker(rel_cb):
    issues = get_issues()
    relevant = filter_relevant_issues(issues, rel_cb)
    
    reqs = iie.IssuetrackerParser(["URS", "FRS", "SDS"]).parse_all(relevant.values())
    return reqs, relevant


In [12]:
def paste_data(ws, data):
    cells = ws.Cells
    cr = cells.Range
    
    di = 2
    hi = 3
    
    header_start = cr("A1")
    
    frs_start = cr("A2")
    frs_end = header_start.Offset(len(data), 1)

    id_start = frs_start.Offset(1, 3)
    id_end = frs_end.Offset(1, 3)
    

    paste_start = header_start
    paste_end = frs_end.Offset(1, len(data[0]))

    xl = ws.Application
    
    with screen_lock(xl):
        print("Pasting test data")
        paste_range = cr(paste_start, paste_end)
        paste_range.Clear()
        cr(id_start, id_end).NumberFormat = "@"
        cr(paste_range.Cells(1, 5), paste_range.Cells(len(data), 5)).NumberFormat = "@"
        paste_range.Value2 = data         

        # This has to come before IndentLevel is set,
        # or it gets fucked up for some reason even
        # though it seems to work correctly when performing
        # the operation manually
        print("Applying alignment formatting")
        col = paste_range.Columns(2)
        col.ColumnWidth = 100
        col.VerticalAlignment = xlc.xlTop
        paste_range.Columns(2).WrapText = True
        
        # Vertical alignment should be top for all
        for i in range(1, len(data[0])+1):
            paste_range.Columns(i).VerticalAlignment = xlc.xlTop
            paste_range.Columns(i).HorizontalAlignment = xlc.xlLeft
        
        # Create a new range to iterate over
        # this improves performance by minimizing the number of 
        # calls that have to be across the COM server to apply
        # the indents
    
        print("Applying row formatting")
        indent_range = cr(paste_start, paste_start.Offset(len(data), 2))
        for i, (d, row) in enumerate(zip(data, indent_range.Rows), 1):
            count = d[0].count(".") * 2
            row.IndentLevel = count
            if count == 0:
                rint = paste_range.Rows(i).Interior
                rint.Pattern = xlc.xlSolid
                rint.PatternColorIndex = xlc.xlAutomatic
                rint.ThemeColor = xlc.xlThemeColorAccent6
                rint.TintAndShade = 0.6
                
        # Document header row formatting
        rint = paste_range.Rows(1).Interior
        rint.Pattern = xlc.xlSolid
        rint.PatternColorIndex = xlc.xlAutomatic
        rint.ThemeColor = xlc.xlThemeColorDark1
        rint.TintAndShade = -0.249977111117893
        
        print("Applying column autofit")
        # fit after filter to account for width of filter icon
        for i in (1, 3, 4, 5):
            paste_range.Columns(i).AutoFit()

        print("Applying row autofit")
        paste_range.Rows.AutoFit()

In [13]:
from officelib.xllib import *
import re

# unlike the Reference class's item key, the key for the Node class
# is (TYPE+FIRSTNUM, OTHERNUMS). e.g. ('URS123', '4.5') instead of
# ('URS', '123.4.5')
_imkey = re.compile(r"(%s)(\d+)\.?([\d\.]*)" % "|".join(['URS', 'FRS', 'SDS'])).match

def key_match(key):
    m = _imkey(key)
    if m:
        type, first, others = m.groups()
        return type+first, others
    return key, ""

def build_frs_tree(all_items, type):
    root = FRS.Root(key_match)
    for frs, req in all_items.items():
        if req.type != type:
            continue
        node = root.add(frs, int(req.obs))
        node.text = req.text
        node.refs = req.refs
        node.milestone = req.milestone
        node.priority = req.priority
    return root

def make_paste_data2(reqs, typ):
    data = [["URS Number", "Text", "References", "Criticality", "Release Version", "Sort Order"]]
    seen = {}
    thekey = lambda req: [int(x or 0) for x in req.num.split(".")]
    reqs = [r for r in reqs if r.type == typ]
    for req in sorted(reqs, key=thekey):
        rkey = req.type + req.num.split(".")[0]
        if rkey in seen:
            so = "=R[-1]C + 1"
        else:
            seen[rkey] = so = (len(seen) + 1) * 1000
        data.append((req.tag, numberify(req.text), "\n".join(req.refs), req.priority, req.milestone, so))
    return data

import re
_numberify_sub = re.compile(r"^(#+|\*+)", flags=re.MULTILINE).sub

def numberify(text):
    
    stack = None
    last = None

    def _numberify(m):
        nonlocal last, stack
        s = m.group(1)
        if s[0] == "*":
            return "  " * len(s) + "*"
        if last is None:
            stack = [1]
            last = s
            lastn = 1
            return "  1)"
        else:
            if len(s) < len(last):
                stack.pop()
            elif len(s) > len(last):
                stack.append(0)
            last = s
            stack[-1] += 1
            return "  " + ".".join(map(str, stack)) + ")"
    return _numberify_sub(_numberify, text)

def main(type='URS', rel_cb=lambda x: True):

    issues = get_issues()
    relevant = filter_relevant_issues(issues, rel_cb)
    reqs = iie.IssuetrackerParser(["URS", "FRS", "SDS"]).parse_all(relevant.values())

    xl = Excel()
    with screen_lock(xl):
        print("Compiling data for final matrix")
        data = make_paste_data2(reqs, type)  
        ws = FRS.get_matrix_sheet(xl)
        paste_data(ws, data)
        print("Done")

In [14]:
_ignore = {3194, 3287}
#(globals().get("_cache", None) or {}).clear()
def relevant(i):
    return (i.sprint_milestone == "Legacy" or i.sprint_milestone == '3.0') and i.id not in _ignore and i.status != "Rejected"

trace_path = 'C:\\Users\\Nathan\\Documents\\Dropbox\\FRS'
user_tests = 'FRS Tests 181127.xlsx'
p1 = os.path.join(trace_path, user_tests)
main('URS', relevant)

Compiling data for final matrix
Pasting test data
Applying alignment formatting
Applying row formatting
Applying column autofit
Applying row autofit
Done


In [68]:
xl = Excel()
while True:
    try:
        wb = xl.Workbooks(1)
        wb.Close(False)
    except:
        break
xl.Quit()
del xl

In [117]:
def rowify():
    try:
        xl
    except NameError:
        xl = Excel()
    rows = xl.Selection.Value2
    out = []
    for tag, text in rows:
        tag = "*" * tag.count(".") + " " + "*" + tag + "*"
        
        out.append(tag + " " + text)
    return "\n".join(out)

In [119]:
print(rowify())

* *URS3910.4* All report types will use a standard file format such that the data can be imported into Microsoft Excel or LibreOffice.
* *URS3910.5* Users will be able to generate reports from both active and archived databases.
* *URS3910.6* Users will be able to clearly identify batches which span multiple databases.
* *URS3910.7* Reports will include metadata that indicates batch name or timespan.
